In [8]:
using Oscar

In [11]:
include("kras_tools.txt");

**Strong Exchange equations**
Let $W=W(B_n) = \langle s_1,\dots,s_n\rangle$ be the $B_n$ root system. The minucule parabolic is given by $P_n = \langle s_1,\dots,s_{n-1}\rangle$. The strong exchange equations for $(B_n,P_n)$ are given by:

\begin{align}\label{eq:B_n_strong}
f_{I,J}^{B} = \begin{cases}
\bigoplus_{i \in I\triangle J} 
%(-1)^{l(\sigma_{i,I})+l(\sigma_{j,J})} 
x_{I \triangle i} \odot x_{J \triangle i} \hspace{19.7mm} \text{ if } |I \triangle J | \equiv 0 \mod 2 \\
x_I \odot x_J \oplus \bigoplus_{i \in I\triangle J} 
%(-1)^{l(\sigma_{i,I})+l(\sigma_{j,J})} 
x_{I \triangle i} \odot x_{J \triangle i} \quad \text{ if } |I \triangle J | \equiv 1 \mod 2 ,\end{cases}
\end{align}.

The code below produces the strong exchange equations for minuscule type $B$. It relies on taking symmetric differences of subsets of $[n]=\{1,\dots,n\}$.

In [16]:
#create powerset of [n]
function p_set(n)
    reduce(vcat,[subsets(n,i) for i in 0:n])
end
#takes symmetric differences
function sym_diff(A,B)
    return union(setdiff(A,B),setdiff(B,A))
end

sym_diff (generic function with 1 method)

In [19]:
function Strong_eqs_B(n)
    P = p_set(n)
    S = subsets(P,2)
    X0 = [s for s in S if length(symdiff(s[1],s[2]))%2 == 0]
    X1 = [s for s in S if length(symdiff(s[1],s[2]))%2 == 1]
    R,x = polynomial_ring(QQ,:x=>P)
    xdict = Dict([P[i]=>x[i] for i in 1:length(P)])
    
    
    F0 = [sum([xdict[sort(sym_diff(a[1],[i]))]*xdict[sort(sym_diff(a[2],[i]))] 
                for i in sym_diff(a[1],a[2])]) for a in X0]

 
    F1 = [sum(vcat([xdict[a[1]]*xdict[a[2]]],[xdict[sort(sym_diff(a[1],[i]))]*xdict[sort(sym_diff(a[2],[i]))] 
                for i in sym_diff(a[1],a[2])])) for a in X1]
  
    return unique([m for m in unique(vcat(F0,F1)) if !(length(monomials(m)) == 1)])
end


Strong_eqs_B (generic function with 1 method)

The code below produces the $n$-cube as the convex hull of indicator vectors for elements in $2^{[n]}$.

In [20]:
#indicator vectors
function ind_vec(A,n)
    @req issubset(A,1:n) "A is not a subset of elements of n"
    z = Vector{Int64}(zeros(n))
    for i in A
        z[i] = 1   
    end
    return z
end
#create cube
function make_cube(n)
    return convex_hull([ind_vec(p,n) for p in p_set(n)])
end

make_cube (generic function with 1 method)

## The $n=3$ case

First we exhibit a height function in the $(B_3,P_3)$ case that does not satisfy the tropical Fink equation, but induces a subdivision of the $3$-cube into strong exchange cells. 

The equation we get in this case is of the form

$$\text{Trop}_{F}(B,3)=\text{min}(x_{\emptyset}+x_{[3]},x_{1}+x_{23},x_{2}+x_{13},x_{3}+x_{12}).$$

In [17]:
F3 = Strong_eqs_B(3)

1-element Vector{QQMPolyRingElem}:
 x[Int64[]]*x[[1, 2, 3]] + x[[1]]*x[[2, 3]] + x[[2]]*x[[1, 3]] + x[[3]]*x[[1, 2]]

We create the $3$-cube and compute the vertices below. We will use this data later.

In [22]:
C3 = make_cube(3)
V = vertices(C3);

**A strong valuation which is not in the Coxeter Dressian**

In [38]:
z = [0, 0, 0,-1, 1,1,2,0];

We show that $z$ induces a strong exchange subdivision.

In [39]:
Sz = subdivision_of_points(C3,z)
subdivision_is_strong(Sz,V,F3)

true

We show that $z$ is not in the tropical hypersurface define by $F3$.

In [41]:
in_prevariety(z,F3)

false

**Study of $\text{Dr}(B_3,P_3)$**

First we study $\text{Dr}(B_3,P_3)$. We observe that the fan structure is too coarse for a study of valuations. We can see this via the $f$-vector, and the fact that the lineality space induces nontrivial strong subdivisions of the cube.

In [23]:
Dr33 = tropical_prevariety(F3)

Polyhedral complex in ambient dimension 8

In [24]:
f_vector(Dr33)

8-element Vector{Int64}:
 0
 0
 0
 0
 0
 1
 4
 6

In [29]:
L33 = lineality_space(Dr33);

In [26]:
S = subdivision_of_points(C3,L33[2])

Subdivision of points in ambient dimension 3

In [28]:
subdivision_is_strong(S,V,F3)

true

**The strong exchange fan and the augmented Coxeter Dressian**

We compute and compare two fans for the B3 case.

1) The *strong exchange fan*: The subfan of the secondary fan of the cube whose cones induce strong exchange subdivisions.

2) The *refined strong Dressian*: The fan obtained by tropicalizing the equation above and then taking the common refinement with the secondary fan of the $3$-cube. We denote this by $\text{Dr}^{\prime}(B_3,P_3)$

**Compute strong exchange fan**

We compute the subfan of the secondary fan inducing strong exchange subdivisions.

In [42]:
Sf = normal_fan(secondary_polytope(C3))#compute secondary fan

Polyhedral fan in ambient dimension 8

In [31]:
Rs = rays_modulo_lineality(Sf)[1];
L = rays_modulo_lineality(Sf)[2];

In [33]:
Cs = cones(Sf);#all cones as incidence matrix
Inds = [[j for j in 1:ncols(Cs) if Cs[i,j] == 1] for i in 1:nrows(Cs)];#convert cones to sets

In [35]:
#pick out cones whose interior corresponds to strong exchange subdivisions
X = []
for i in 1:length(Inds)-1#omit empty cone
    r = sum([Rs[j] for j in Inds[i]])
    S = subdivision_of_points(C3,r)
    if subdivision_is_strong(S,V,F3)
        push!(X,Inds[i])
    end
end

In [49]:
#saving fan
sFan3 = polyhedral_fan(incidence_matrix(Vector{Vector{Int64}}(union(X,[]))),Rs,L)

Polyhedral fan in ambient dimension 8

**Compute refined Dressian**

We compute $\text{Dr}^{\prime}(B_3,P_3)$ by taking the common refinement of $Dr(B_3,P_3)$ with the secondary fan of the $3$-cube.

In [45]:
Sfc = polyhedral_complex(Sf)

Polyhedral complex in ambient dimension 8

In [48]:
rDr33 = common_refinement(Sfc,Dr33)

Polyhedral complex in ambient dimension 8

**Comparison of fans as complexes**

**Dimensions**

In [51]:
dim(sFan3)

8

In [52]:
dim(rDr33)

7

In [56]:
is_pure(sFan3)

false

In [57]:
is_pure(rDr33)

true

**$f$-vectors**

In [53]:
f_vector(sFan3)

8-element Vector{ZZRingElem}:
 0
 0
 0
 0
 14
 40
 32
 2

In [54]:
f_vector(rDr33)

8-element Vector{Int64}:
  0
  0
  0
  0
  1
 16
 48
 36

**Containment of Dressian in Strong exchange fan**

We show below that every cone in the strong exchange fan has a representative in the Coxeter Dressian $\text{Dr}(B_3,P_3)$.

In [61]:
#Show that the baricenter of each cone is contained in the Dressian
J = []
for i in 1:length(X)
    r = sum([Rs[j] for j in X[i]])
    if in_prevariety(r,F3)
        push!(J,X[i])
    end
end
J;

In [62]:
J == X

true

## The $n = 4$ case

The strong exchange equations in $n=4$ are listed below.

The following $10$ equations correspond to facets of the $4-cube$ which are each isomorphic to $3$-cubes.
\begin{align*}
f_{1,1234} &= x_1 \odot x_{1234} \oplus x_{12} \odot x_{134} \oplus x_{13} \odot x_{124} \oplus x_{14} \odot x_{123}\\
f_{\emptyset,234} &= x_{\emptyset} \odot x_{234} \oplus x_{2} \odot x_{34} \oplus x_{3} \odot x_{24} \oplus x_{4} \odot x_{23}\\
f_{2,1234} &= x_2 \odot x_{1234} \oplus x_{12} \odot x_{234} \oplus x_{23} \odot x_{124} \oplus x_{24} \odot x_{123}\\
f_{\emptyset,134} &= x_{\emptyset} \odot x_{134} \oplus x_{1} \odot x_{34} \oplus x_{3} \odot x_{14} \oplus x_{4} \odot x_{13}\\
f_{3,1234} &= x_3 \odot x_{1234} \oplus x_{13} \odot x_{234} \oplus x_{23} \odot x_{134} \oplus x_{34} \odot x_{123}\\
f_{\emptyset,124} &= x_{\emptyset} \odot x_{124} \oplus x_{1} \odot x_{24} \oplus x_{2} \odot x_{14} \oplus x_{4} \odot x_{12}\\
f_{4,1234} &= x_4 \odot x_{1234} \oplus x_{14} \odot x_{234} \oplus x_{24} \odot x_{134} \oplus x_{34} \odot x_{124}\\
f_{\emptyset,123} &= x_{\emptyset} \odot x_{123} \oplus x_{1} \odot x_{23} \oplus x_{2} \odot x_{13} \oplus x_{3} \odot x_{12}\, .
\end{align*}
Moreover, there are two equations whose support is not contained in a proper face:
\begin{align*}
f_{\emptyset,1234}&= x_1 \odot x_{234} \oplus x_{2} \odot x_{134} \oplus x_{3} \odot x_{124} \oplus x_{4} \odot x_{123}\\
f_{1,234}&=x_{\emptyset} \odot x_{1234} \oplus x_{12} \odot x_{34} \oplus x_{13} \odot x_{24} \oplus x_{14} \odot x_{23}\, .
\end{align*}

We study $\text{Dr}(B_4,P_4)$, and another prevariety with the same support, but it is computed only using the $n=3$ facets. In other words, the second prevariety only uses strong exchange equations with $4$ monomials. One of our results is that these objects have the same support.

Since these computations take a long time, we precompute the relevant objects.

In [63]:
Dr44 = load("Coxeter_Dressian_B4.mrdi")

Polyhedral complex in ambient dimension 16

In [64]:
F44 = load("Coxeter_prevariety_faces_B4.mrdi")

Polyhedral complex in ambient dimension 16

In [65]:
f_vector(Dr44)

12-element Vector{Int64}:
    0
    0
    0
    0
    0
    0
    1
   36
  280
  960
 1540
  912

In [66]:
f_vector(F44)

12-element Vector{Int64}:
    0
    0
    0
    0
    0
    0
    1
   36
  280
  960
 1528
  900